In [1]:
%%capture
!pip -q install git+https://github.com/huggingface/transformers
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install faiss-gpu
!pip install einops
!pip install --upgrade --quiet  langchain-community chromadb bs4 qdrant-client
!pip install langchainhub
!pip install sentence-transformers
!pip install tqdm
!pip install rouge_score nltk bert_score

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/MyDrive/DATASCI266/project')

Mounted at /content/drive


In [3]:
import os
import torch
import warnings
from datasets import load_dataset
from tqdm import tqdm
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain import PromptTemplate, LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer , AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline, BitsAndBytesConfig
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

In [4]:
from summary_utils import SummaryEvaluator, DatasetManager, SummaryModel

In [5]:
# Evaluator
summary_evaluator = SummaryEvaluator()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Load the XSum dataset
data_manager = DatasetManager(sample_size=200)
dataset = data_manager.load_sampled_dataset(dataset_label="test")

# Initialize the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
t5model = TFT5ForConditionalGeneration.from_pretrained("t5-base")
t5tokenizer = T5Tokenizer.from_pretrained("t5-base")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
t5_pipe = pipeline(
    "summarization",
    model=t5model,
    tokenizer=t5tokenizer,
    max_length=1000,
    truncation=True,
    device=0
)
#t5_pipe.model.config.pad_token_id = t5_pipe.model.config.eos_token_id

In [10]:
t5_llm_lc = HuggingFacePipeline(pipeline=t5_pipe)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [11]:
class RAG_Summarization_Model:
  def __init__(self, chunk_size=200, chunk_overlap=50):
    self.vector_store = None
    self.rag_chain = None
    self.chunk_size = chunk_size
    self.chunk_overlap = chunk_overlap

  def fit(self, dataset):
    # Function to split documents and include metadata
    def create_documents_with_metadata(dataset, chunk_size=200, chunk_overlap=50):
      # Initialize the text splitter
      text_splitter = RecursiveCharacterTextSplitter(
          chunk_size=chunk_size,  # Adjust based on your needs
          chunk_overlap=chunk_overlap  # Adjust based on your needs
      )
      documents = []
      for example in dataset:
          text = example['document']
          # Split the text into chunks while including metadata
          chunks = text_splitter.split_text(text)
          for chunk in chunks:
              documents.append(Document(page_content=chunk, metadata={"id": example['id']}))
      return documents
    # Create documents with metadata
    documents = create_documents_with_metadata(dataset, self.chunk_size, self.chunk_overlap)
    # Create a FAISS vector store
    self.vector_store = FAISS.from_documents(documents, embeddings)

    rag_template = """summarize: {context}"""

    rag_prompt = ChatPromptTemplate.from_template(rag_template)

    self.rag_chain = (
        {"context": lambda inputs: self.format_docs(self.query_vector_store(inputs['query'], inputs['id'], k=inputs['k'], fetch_k=inputs['fetch_k'])),
        "question": RunnablePassthrough()}
        | rag_prompt
        | t5_llm_lc
    )

  def query_vector_store(self, query_text, summary_id, k=3, fetch_k=20):
    # Embed the query text
    query_vector = embeddings.embed_query(query_text)
    # Search the vector store
    docs = self.vector_store.similarity_search_by_vector(query_vector, filter=dict(id=summary_id), k=k, fetch_k=fetch_k)
    return docs

  def format_docs(self, docs):
      return "\n\n".join(doc.page_content for doc in docs)

  def predict(self, dataset, k, fetch_k):
    results = []
    i = 1
    for example in tqdm(dataset):
        print(i)
        text = example['document']
        sentences = re.split(r'[.?!]\s+', text)
        example_query = sentences[0] + ". " + sentences[-1]
        #summary_w_prompt = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        summary = self.rag_chain.invoke({"query": example_query, "id": example['id'], "k":k, 'fetch_k':fetch_k})
        #summary = summary_w_prompt.split("summarize: ",1)[1]
        results.append(summary)
        i += 1
    return results

In [12]:
references = [example['summary'] for example in dataset]

In [13]:
import re

In [14]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=50)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

  0%|          | 0/200 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 1000, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


1


  0%|          | 1/200 [00:35<1:57:17, 35.36s/it]Your max_length is set to 1000, but your input_length is only 60. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


2


  1%|          | 2/200 [00:49<1:15:13, 22.79s/it]Your max_length is set to 1000, but your input_length is only 126. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=63)


...

 99%|█████████▉| 198/200 [1:04:31<00:38, 19.01s/it]Your max_length is set to 1000, but your input_length is only 75. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=37)


199


100%|█████████▉| 199/200 [1:04:45<00:17, 17.71s/it]Your max_length is set to 1000, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)


200


100%|██████████| 200/200 [1:05:02<00:00, 19.51s/it]


In [15]:
print("chunk_size=200, chunk_overlap=50")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=50
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.19298711667785692, 'rouge2': 0.02553829295155474, 'rougeL': 0.13424432362532607}
Average BLEU score: 0.016650877182075922


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5076683163642883
Average VECTOR_SIMILARITY score: 0.4748822748661041
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.19664720515859652, 'rouge2': 0.02759882106213783, 'rougeL': 0.1332688088445275}
Average BLEU score: 0.015191306880427207


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5078788995742798
Average VECTOR_SIMILARITY score: 0.47750091552734375


{'rouge': {'rouge1': 0.19664720515859652,
  'rouge2': 0.02759882106213783,
  'rougeL': 0.1332688088445275},
 'bleu': 0.015191306880427207,
 'bertscore': 0.5078788995742798,
 'vector_similarity': 0.47750091552734375}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=200, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

In [17]:
print("chunk_size=200, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=200, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.1950831554896836, 'rouge2': 0.026155660647287974, 'rougeL': 0.13444095791481897}
Average BLEU score: 0.016656884429856446


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5023991465568542
Average VECTOR_SIMILARITY score: 0.4462801218032837
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.1950786207420211, 'rouge2': 0.029222043000894857, 'rougeL': 0.13764625211684473}
Average BLEU score: 0.016580938615940972


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5061159729957581
Average VECTOR_SIMILARITY score: 0.460028737783432
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.19956988683082308, 'rouge2': 0.029280050850381677, 'rougeL': 0.13801332127503035}
Average BLEU score: 0.015987795550274568


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5087863802909851
Average VECTOR_SIMILARITY score: 0.4747479557991028
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.19826932173786463, 'rouge2': 0.027519444493321304, 'rougeL': 0.1355098901040085}
Average BLEU score: 0.01526413955072173


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5079150199890137
Average VECTOR_SIMILARITY score: 0.4847949147224426


{'rouge': {'rouge1': 0.19826932173786463,
  'rouge2': 0.027519444493321304,
  'rougeL': 0.1355098901040085},
 'bleu': 0.01526413955072173,
 'bertscore': 0.5079150199890137,
 'vector_similarity': 0.4847949147224426}

In [ ]:
rag_model = RAG_Summarization_Model(chunk_size=150, chunk_overlap=0)
rag_model.fit(dataset)
predictions1 = rag_model.predict(dataset, k=3, fetch_k=50)
predictions2 = rag_model.predict(dataset, k=5, fetch_k=50)
predictions3 = rag_model.predict(dataset, k=10, fetch_k=50)
predictions4 = rag_model.predict(dataset, k=15, fetch_k=50)

In [19]:
print("chunk_size=150, chunk_overlap=0")
print("k=3, fetch_k=50")
summary_evaluator.evaluate(references, predictions1)
print("k=5, fetch_k=50")
summary_evaluator.evaluate(references, predictions2)
print("k=10, fetch_k=50")
summary_evaluator.evaluate(references, predictions3)
print("k=15, fetch_k=50")
summary_evaluator.evaluate(references, predictions4)

chunk_size=150, chunk_overlap=0
k=3, fetch_k=50
Average ROUGE score: {'rouge1': 0.18751923349254207, 'rouge2': 0.02489593430180183, 'rougeL': 0.13291008914878907}
Average BLEU score: 0.017013773622862036


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.49755218625068665
Average VECTOR_SIMILARITY score: 0.4476458728313446
k=5, fetch_k=50
Average ROUGE score: {'rouge1': 0.19227867817004532, 'rouge2': 0.027432241459765628, 'rougeL': 0.1318679612433241}
Average BLEU score: 0.016377802886913706


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5040209889411926
Average VECTOR_SIMILARITY score: 0.46197113394737244
k=10, fetch_k=50
Average ROUGE score: {'rouge1': 0.19592475103870385, 'rouge2': 0.029157489153483193, 'rougeL': 0.13431047939073004}
Average BLEU score: 0.015928235312154773


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.508174479007721
Average VECTOR_SIMILARITY score: 0.4780488908290863
k=15, fetch_k=50
Average ROUGE score: {'rouge1': 0.19298134108458315, 'rouge2': 0.028497367623662843, 'rougeL': 0.13085679545726184}
Average BLEU score: 0.015651171441174883


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

Average BERTSCORE score: 0.5083100199699402
Average VECTOR_SIMILARITY score: 0.478331595659256


{'rouge': {'rouge1': 0.19298134108458315,
  'rouge2': 0.028497367623662843,
  'rougeL': 0.13085679545726184},
 'bleu': 0.015651171441174883,
 'bertscore': 0.5083100199699402,
 'vector_similarity': 0.478331595659256}